In [1]:
import numpy as np
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import *
import qiskit

In [42]:
num_qubits = 32

In [79]:
def create_random_states():

    alice_state = np.random.randint(0, 2, size=num_qubits)
    bob_basis = np.random.randint(0, 2, size=num_qubits)
    noise_prob = 0.5
    print(f"Alice's State:\t {np.array2string(alice_state, separator='')}")
    print(f"Bob's Basis:\t {np.array2string(bob_basis, separator='')}")
    
    return alice_state, bob_basis, noise_prob

In [95]:
# function to define the circuit for an exchange of bit string
def make_b92_circ(enc_state, meas_basis, noise_prob):
    
    num_qubits = len(enc_state)

    b92 = QuantumCircuit(num_qubits)

    for index in range(len(enc_state)):
        if enc_state[index] == 1:
            b92.h(index)

    b92.barrier()
    
    noise = []

    for index in range(len(enc_state)):
        if np.random.random() <= noise_prob:
            b92.x(index)
            noise.append(1)
        else:
            noise.append(0)
            
            
    print(f"Noise:\t\t {np.array2string(np.array(list(noise), dtype=int), separator='')}")
    
    b92.barrier()  

    for index in range(len(meas_basis)):
        if meas_basis[index] == 1:
            b92.h(index)

    # Do not change below this line
    b92.measure_all()
    return b92

In [96]:
def b92_sifting(enc_state, meas_basis, meas_result):
    
    num_qubits = len(enc_state)
    
    sender_key = ''
    receiver_key = ''
    
    for i in range(len(meas_result)):
      if meas_result[i] == '1':
        sender_key+=str(enc_state[i])
        receiver_key+=str((meas_basis[i]+1)%2)
        
    return (sender_key, receiver_key)

In [100]:
key_list = []
iterations = 1

for index in range (0,iterations):

    alice_state, bob_basis, noise_prob = create_random_states()

    #Alice sends the bits to Bob but Eve eavesdrops in between
    b92_circ = make_b92_circ(alice_state, bob_basis, noise_prob)

    #final result got by Eve after measurement
    bob_res = execute(
            b92_circ.reverse_bits(),
            backend=QasmSimulator(),
            shots=1, 
            seed_simulator=10
        ).result().get_counts().most_frequent()

    print(f"Bob's result:\t [{bob_res}]")
    # key sifting takes place to get the final key to be used
    alice_key, bob_key = b92_sifting(alice_state, bob_basis, bob_res)
    
#     print(f"Alice's final key: \t{alice_key}")
#     print(f"Bob's final key: \t{bob_key}")
    key_list.append(bob_key)
#     print(f"Final key: {bob_key} ")
    
avg_key_length = sum( map( len, key_list))/ len( key_list)
print(f"Initial key length = {num_qubits}")
print(f"Final average key length after {iterations} iterations =  {avg_key_length}")

Alice's State:	 [01001101111001010000111111110100]
Bob's Basis:	 [01111101110010100111011001111100]
Noise:		 [01000111110100011010110011110001]
Bob's result:	 [00100010001110011100100010001001]
Initial key length = 32
Final average key length after 1 iterations =  12.0
